In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import warnings

In [2]:
pred_dir = Path('./pck/masactrl_controlnet/edit_openpose_keypoints') # directory including keypoints.json files
gt_dir   = Path('./pck/pose_selected') # directory including keypoints.json files
person_idx = 0   # Person index (0-base)

pred_files = sorted(pred_dir.glob('*.json'))
gt_files   = sorted(gt_dir.glob('*.json'))*10
pair_cnt = min(len(pred_files), len(gt_files))
if len(pred_files) != len(gt_files):
    warnings.warn(f'Directory sizes differ (pred {len(pred_files)}, gt {len(gt_files)}). Evaluating {pair_cnt} pairs.')

In [3]:
import json
import numpy as np

def count_kpts(json_path):
    with open(json_path) as f:
        data = json.load(f)
    if isinstance(data, list):
        data = data[0]
    arr = np.asarray(data["people"][0]["pose_keypoints_2d"])
    return arr.size // 3

print("GT  :", count_kpts(gt_files[0]))
print("Pred:", count_kpts(pred_files[0]))

GT  : 18
Pred: 18


In [4]:
results = []

pred_w, pred_h = 512, 512      # prediction image resolution
thresh_torso, thresh_head = 0.2, 0.5

for idx in range(pair_cnt):
    pred_path, gt_path = pred_files[idx], gt_files[idx]
    try:
        # JSON load (handle list wrapper)
        with open(pred_path) as f:
            pred_data = json.load(f)
        with open(gt_path) as f:
            gt_data = json.load(f)
        if isinstance(pred_data, list):
            pred_data = pred_data[0]
        if isinstance(gt_data, list):
            gt_data = gt_data[0]

        # people extraction
        pred_people = pred_data["people"]
        gt_people = gt_data["people"]
        if not pred_people or not gt_people:
            raise ValueError("no people")
        if person_idx >= len(pred_people) or person_idx >= len(gt_people):
            raise IndexError("person_index out of range")

        pred_arr = np.asarray(pred_people[person_idx]["pose_keypoints_2d"]).reshape(-1, 3)
        gt_arr = np.asarray(gt_people[person_idx]["pose_keypoints_2d"]).reshape(-1, 3)

        n_common = min(len(pred_arr), len(gt_arr))
        kp_pred = pred_arr[:n_common, :2]
        kp_gt = gt_arr[:n_common, :2]
        conf_gt = gt_arr[:n_common, 2]

        # scale GT (768×768 → 512×512)
        gt_w, gt_h = gt_data.get("canvas_width"), gt_data.get("canvas_height")
        if gt_w is None or gt_h is None:
            raise ValueError("GT canvas size missing")
        kp_gt *= [pred_w / gt_w, pred_h / gt_h]

        # choose indices per format
        if n_common >= 25:          # BODY 25
            idx_LS, idx_RH = 5, 9
            head_pairs = [(17, 18), (15, 16), (0, 1)]
        elif n_common == 18:        # BODY 18
            idx_LS, idx_RH = 5, 12
            head_pairs = [(4, 3), (2, 1), (0, 17)]
        elif n_common == 17:        # COCO 17
            idx_LS, idx_RH = 5, 12
            head_pairs = [(4, 3), (2, 1)]
        else:
            raise ValueError("unexpected keypoint count")

        # torso length
        torso_len = np.linalg.norm(kp_gt[idx_LS] - kp_gt[idx_RH])
        if torso_len == 0:
            raise ValueError("zero torso length")

        # head length
        head_len = None
        for a, b in head_pairs:
            if a < n_common and b < n_common and conf_gt[a] > 0.05 and conf_gt[b] > 0.05:
                d = np.linalg.norm(kp_gt[a] - kp_gt[b])
                if d > 0:
                    head_len = d
                    break
        if head_len is None:
            raise ValueError("cannot determine head length")

        # distances & correctness
        valid = conf_gt > 0.05
        dists = np.linalg.norm(kp_pred[valid] - kp_gt[valid], axis=1)
        correct_torso = dists < thresh_torso * torso_len
        correct_head = dists < thresh_head * head_len

        results.append({
            "index": idx,
            "pred_file": pred_path.name,
            "gt_file": gt_path.name,
            "PCK@0.2": float(correct_torso.mean()),
            "PCKh@0.5": float(correct_head.mean())
        })

    except Exception as e:
        results.append({
            "index": idx,
            "pred_file": pred_path.name,
            "gt_file": gt_path.name,
            "PCK@0.2": np.nan,
            "PCKh@0.5": np.nan,
            "error": str(e)
        })

In [5]:
valid_pck = [r["PCK@0.2"] for r in results if isinstance(r["PCK@0.2"], float) and not np.isnan(r["PCK@0.2"])]
mean_pck_list = np.mean(valid_pck) if valid_pck else float("nan")
print(f"\nMean PCK@0.2 = {mean_pck_list:.4f}")


Mean PCK@0.2 = 0.9111


In [6]:
valid_pck = [r["PCKh@0.5"] for r in results if isinstance(r["PCKh@0.5"], float) and not np.isnan(r["PCKh@0.5"])]
mean_pck_list = np.mean(valid_pck) if valid_pck else float("nan")
print(f"\nMean PCKh@0.5 = {mean_pck_list:.4f}")


Mean PCKh@0.5 = 0.8889
